# Imports and Installs

In [7]:
import matplotlib.pyplot as plt

from copy import deepcopy
from tabulate import tabulate

# Data

In [8]:
data_p1 = {1: "GTAATGGACGT",
           2: "GCGTTGCGT",
           3: "GCATCACGCAAT",
           4: "GTGCTACGCCG",
           5: "GGGATGCACG"
           }

data_p2 = {1: "GCCCCTA",
           2: "CTAGCCTCA",
           3: "CTACGCTAA",
           4: "CGAGCTTA",
           5: "CGCGACT"
           }
data_p3 = {1: "GCAGCTA",
           2: "TAGCTCA",
           3: "TACGTTA",
           4: "CGAGCTTA"}

# Needleman-Wunsch Global Alignment

In [9]:
def get_score(table_score, c1, c2, s_match, s_nomatch):
    if c1 == c2:
        return table_score + s_match
    else:
        return table_score + s_nomatch

In [10]:
def plot_table(table, s1, s2):

    # Convert data to strings
    cell_text = []
    for row in table:
        cell_text.append([])
        for val in row:
            cell_text[-1].append("%d"%(val["val"]))

    # Get row labels
    row_labels = []
    for c in s1:
        row_labels.append(c)

    # Get column labels
    col_labels = []
    for c in s2:
        col_labels.append(c)

    # Hide graph
    fig, ax = plt.subplots()
    fig.patch.set_visible(False)
    ax.axis("off")
    ax.axis("tight")

    # Display table
    ax.table = plt.table(cellText=cell_text,
                         rowLabels=row_labels,
                         rowLoc="center",
                         colLabels=col_labels,
                         colLoc="center",
                        #  colWidths=[0.5, 0.5],
                         cellLoc="center"
                         )
    # fig.tight_layout()
    plt.show()

In [11]:
def needleman_wunsch(s1, s2, s_match=2, s_nomatch=-2, display=False):

    # Add -
    s1 = "-" + s1
    s2 = "-" + s2

    # Set up table
    table = []
    for i in range(len(s1)):
        table.append([])
        for j in range(len(s2)):
            table[-1].append({"val": None, "from": None})

    # Initilize table's first row and column
    table[0][0]["val"] = 0
    for i in range(1, len(s1)):
        table[i][0]["val"] = table[i-1][0]["val"] + s_nomatch
        table[i][0]["from"] = [i-1, 0]
    for j in range(1, len(s2)):
        table[0][j]["val"] = table[0][j-1]["val"] + s_nomatch
        table[0][j]["from"] = [0, j-1]

    # Compute table
    for i in range(1, len(s1)):
        for j in range(1, len(s2)):

            # Calculate values from surrounding cells
            sorter = []
            sorter.append({"val": get_score(table[i-1][j]["val"], s1[i], s2[j], s_match, s_nomatch), "from": [i-1, j]})
            sorter.append({"val": get_score(table[i][j-1]["val"], s1[i], s2[j], s_match, s_nomatch), "from": [i, j-1]})
            sorter.append({"val": get_score(table[i-1][j-1]["val"], s1[i], s2[j], s_match, s_nomatch), "from": [i-1, j-1]})

            # Select and keep best score
            sorter = sorted(sorter, key=lambda x: x["val"], reverse=True)
            table[i][j]["val"] = sorter[0]["val"]
            table[i][j]["from"] = []
            for row in sorter:
                if row["val"] == table[i][j]["val"]:
                    table[i][j]["from"].append(row["from"])

    # Display
    if display == 1:
        table_new = []
        for i in range(len(s1)):
            table_new.append(["%s - %d"%(s1[i], i)])
            for j in range(len(s2)):
                table_new[-1].append(table[i][j])
        h = [-1] + ["%s - %d"%(s2[j], j) for j in range(len(s2))]
        print(tabulate(table_new, tablefmt="plain", headers=h))

        # plot_table(table, s1, s2)

    if display == 2:
        table_new = []
        for i in range(len(s1)):
            table_new.append(["%s  %d"%(s1[i], i)])
            for j in range(len(s2)):
                table_new[-1].append(table[i][j]["val"])
        h = [-1] + ["%s  %d"%(s2[j], j) for j in range(len(s2))]
        print(tabulate(table_new, tablefmt="grid", headers=h))

    return table, table[len(s1)-1][len(s2)-1]["val"]

# ClustalW Alignment

In [12]:
def get_pairwise_table(sequences, s_match=2, s_nomatch=-2, mirror=False, display=False):

    # Set up table
    table = [["corner"]]
    for k1 in sequences:
        table.append([k1])
        table[0].append(k1)
        for k2 in sequences:
            table[-1].append(None)

    # Get pairwise alignment scores
    for i, k1 in enumerate(sequences):
        for j, k2 in enumerate(sequences):
            if k1 != k2:
                _, score = needleman_wunsch(sequences[k1], sequences[k2], s_match=s_match, s_nomatch=s_nomatch)
                if i < j:
                    table[i+1][j+1] = score
                elif mirror:
                    table[i+1][j+1] = score

    # Display
    if display:
        print(tabulate(table, tablefmt="grid"))

    return table

In [13]:
def clustalw(sequences, s_match=2, s_nomatch=-2, display=False):
    seqs = deepcopy(sequences)

    # Get the initial table
    table = get_pairwise_table(seqs, s_match=s_match, s_nomatch=s_nomatch)

    # Iterate until the table is empty
    while len(table) > 2 and len(table[0]) > 2:

        # Get min score on table
        min_s = 1000000000000000
        min_i = None
        min_j = None
        for i in range(1, len(table)):
            for j in range(1, len(table[i])):
                if i < j:
                    if table[i][j] < min_s:
                        min_s = table[i][j]
                        min_i = i
                        min_j = j

        # Record the current pair


        # Delete pair from sequences
        del seqs[min_i]
        del seqs[min_j]

        # Get new table
        table = get_pairwise_table(seqs, s_match=s_match, s_nomatch=s_nomatch)
        if display:
            print(tabulate(table, tablefmt="grid"))
            print()

    # Display
    if display:
        print(tabulate(table, tablefmt="grid"))

# clustalw(data_p1, display=True)

# STAR Alignment

In [33]:
def get_center_seq(sequences, s_match=2, s_nomatch=-1, display=False):

    # Get table
    table = get_pairwise_table(sequences, s_match=s_match, s_nomatch=s_nomatch, mirror=True)

    # Compute scores
    max_s = -1111111111111
    max_i = None
    for i in range(1, len(table)):
        s = 0
        for j in range(1, len(table[i])):
            if table[i][j] is not None:
                s += table[i][j]
        table[i].append(s)

        # Get max score
        if s > max_s:
            max_s = s
            max_i = i

    # Update table header
    table[0].append("Score")

    # Display
    if display:
        print(tabulate(table, tablefmt="grid"))

    # Get sequence key
    k = table[max_i][0]

    return table, k

# table, k = get_center_seq(data_p3, s_match=1, s_nomatch=-1, display=True)
# k

In [ ]:
def align_to_center(sequences, s_match=2, s_nomatch=-1):

    # Get center sequence
    _, kc = get_center_seq(sequences, s_match=s_match, s_nomatch=s_nomatch)
    cent = sequences[kc]
    print(cent)

    # Align each sequence to center sequence
    for k in sequences:

        # Get table
        table, score = needleman_wunsch(cent, sequences[k], s_match=s_match, s_nomatch=s_nomatch, display=True)

        # Traverse table
        alignment = []
        prev = table[len(table) - 1][len(table[0]) - 1]["from"]
        while True:
            alignment.insert(0, prev[1])
            curr = table[prev[0]][prev[1]]["from"]
            print(prev, curr)

            prev = deepcopy(curr)

            if curr is None:
                break

        # Get text version of alignment

        print(alignment)

        break


# align_to_center(data_p1)

# Sum-of-Pair Scoring

In [ ]:
# txt = """S1: G---CCCCT-A
# S2: CTAGCC--TCA
# S3: CTA-CGC-TAA
# S4: CGAGC---TTA
# S5: CGCG---ACT-
# """
# txt = txt.split("\n")
# txt = [x.split(": ") for x in txt]

# sp = 0
# for i in range(len(txt[0][1])):
#     s = []
#     for j in range(len(txt)):


# Problem 1

In [ ]:
for i, k1 in enumerate(data_p1):
    for j, k2 in enumerate(data_p1):
        if i < j and k1 != k2:
            print(k1, "-", k2)
            _, _ = needleman_wunsch(data_p1[k1], data_p1[k2], display=1)
            print()

1 - 2
-1      - - 0                          G - 1                           C - 2                                    G - 3                                          T - 4                                          T - 5                                 G - 6                                 C - 7                                         G - 8                                 T - 9
- - 0   {'val': 0, 'from': None}       {'val': -2, 'from': [0, 0]}     {'val': -4, 'from': [0, 1]}              {'val': -6, 'from': [0, 2]}                    {'val': -8, 'from': [0, 3]}                    {'val': -10, 'from': [0, 4]}          {'val': -12, 'from': [0, 5]}          {'val': -14, 'from': [0, 6]}                  {'val': -16, 'from': [0, 7]}          {'val': -18, 'from': [0, 8]}
G - 1   {'val': -2, 'from': [0, 0]}    {'val': 2, 'from': [[0, 0]]}    {'val': 0, 'from': [[1, 1]]}             {'val': 2, 'from': [[1, 2]]}                   {'val': 0, 'from': [[1, 3]]}                   {'val': -2, 'from': [

In [ ]:
txt = """S1: G--T-AATGGACGT
S2: GCGT---TG-C-GT

S1: G--T-AATGG-A-CGT
S3: GCATCA-CG-CAAT–-

S1: G---TAATGGAC-GT
S4: GTGCTA-CG-C-CG-

S1: G--TAATGG-ACGT
S5: GGGA--TG-CACG-

S2: GCGTTGC-----GT
S3: GCAT-C-ACGCAAT

S2: GCGTTGC------GT
S4: G--T-GCTACGCCG-

S2: GCG--TTGC--GT
S5: GG-GAT-GCACG-

S3: G--CATCACGC-AAT
S4: GTGCT-A-CGCCG--

S3: G--CAT-CACGCAA-T
S5: GGGA-TGC----A-CG

S4: G---TGCTACGCCG
S5: GGGATGCA-C---G"""

txt = txt.split("\n\n")
txt = [x.split("\n") for x in txt]
txt

[['S1: G--T-AATGGACGT', 'S2: GCGT---TG-C-GT'],
 ['S1: G--T-AATGG-A-CGT', 'S3: GCATCA-CG-CAAT–-'],
 ['S1: G---TAATGGAC-GT', 'S4: GTGCTA-CG-C-CG-'],
 ['S1: G--TAATGG-ACGT', 'S5: GGGA--TG-CACG-'],
 ['S2: GCGTTGC-----GT', 'S3: GCAT-C-ACGCAAT'],
 ['S2: GCGTTGC------GT', 'S4: G--T-GCTACGCCG-'],
 ['S2: GCG--TTGC--GT', 'S5: GG-GAT-GCACG-'],
 ['S3: G--CATCACGC-AAT', 'S4: GTGCT-A-CGCCG--'],
 ['S3: G--CAT-CACGCAA-T', 'S5: GGGA-TGC----A-CG'],
 ['S4: G---TGCTACGCCG', 'S5: GGGATGCA-C---G']]

In [ ]:
for row in txt:
    s1 = row[0].split(" ")[1]
    s2 = row[1].split(" ")[1]

    n_gap = 0
    mat = 0
    for i in range(len(s1)):
        if s1[i] != "-" and s2[i] != "-":
            n_gap += 1
            if s1[i] == s2[i]:
                mat += 1
    per = 1 - (mat / n_gap)
    print(row[0])
    print(row[1])
    print("Not Gap:", n_gap)
    print("Match:", mat)
    print("Percent:", round(per, 5))
    print()

S1: G--T-AATGGACGT
S2: GCGT---TG-C-GT
Not Gap: 7
Match: 6
Percent: 0.14286

S1: G--T-AATGG-A-CGT
S3: GCATCA-CG-CAAT–-
Not Gap: 8
Match: 5
Percent: 0.375

S1: G---TAATGGAC-GT
S4: GTGCTA-CG-C-CG-
Not Gap: 7
Match: 5
Percent: 0.28571

S1: G--TAATGG-ACGT
S5: GGGA--TG-CACG-
Not Gap: 7
Match: 6
Percent: 0.14286

S2: GCGTTGC-----GT
S3: GCAT-C-ACGCAAT
Not Gap: 7
Match: 4
Percent: 0.42857

S2: GCGTTGC------GT
S4: G--T-GCTACGCCG-
Not Gap: 5
Match: 5
Percent: 0.0

S2: GCG--TTGC--GT
S5: GG-GAT-GCACG-
Not Gap: 6
Match: 5
Percent: 0.16667

S3: G--CATCACGC-AAT
S4: GTGCT-A-CGCCG--
Not Gap: 8
Match: 5
Percent: 0.375

S3: G--CAT-CACGCAA-T
S5: GGGA-TGC----A-CG
Not Gap: 6
Match: 4
Percent: 0.33333

S4: G---TGCTACGCCG
S5: GGGATGCA-C---G
Not Gap: 7
Match: 6
Percent: 0.14286



In [ ]:
table = get_pairwise_table(data_p1, display=True)

+--------+---+----+---+----+----+
| corner | 1 |  2 | 3 |  4 |  5 |
+--------+---+----+---+----+----+
| 1      |   | 12 | 4 | 10 | 16 |
+--------+---+----+---+----+----+
| 2      |   |    | 4 | 10 | 12 |
+--------+---+----+---+----+----+
| 3      |   |    |   |  6 |  8 |
+--------+---+----+---+----+----+
| 4      |   |    |   |    | 16 |
+--------+---+----+---+----+----+
| 5      |   |    |   |    |    |
+--------+---+----+---+----+----+


# Problem 2

In [ ]:
for i, k1 in enumerate(data_p2):
    for j, k2 in enumerate(data_p2):
        if i < j and k1 != k2:
            print(k1, "-", k2)
            _, _ = needleman_wunsch(data_p2[k1], data_p2[k2], display=1, s_match=2, s_nomatch=-1)
            print()

1 - 2
-1     - - 0                        C - 1                                 T - 2                                  A - 3                                  G - 4                                  C - 5                                 C - 6                                 T - 7                          C - 8                                  A - 9
- - 0  {'val': 0, 'from': None}     {'val': -1, 'from': [0, 0]}           {'val': -2, 'from': [0, 1]}            {'val': -3, 'from': [0, 2]}            {'val': -4, 'from': [0, 3]}            {'val': -5, 'from': [0, 4]}           {'val': -6, 'from': [0, 5]}           {'val': -7, 'from': [0, 6]}    {'val': -8, 'from': [0, 7]}            {'val': -9, 'from': [0, 8]}
G - 1  {'val': -1, 'from': [0, 0]}  {'val': -1, 'from': [[0, 0]]}         {'val': -2, 'from': [[1, 1], [0, 1]]}  {'val': -3, 'from': [[1, 2], [0, 2]]}  {'val': -1, 'from': [[1, 3], [0, 3]]}  {'val': -2, 'from': [[1, 4]]}         {'val': -3, 'from': [[1, 5]]}         {'val': -4, 'from':

In [ ]:
table, kc = get_center_seq(data_p2, s_match=2, s_nomatch=-1, display=True)
kc

+--------+----+----+----+----+----+-------+
| corner |  1 |  2 |  3 |  4 |  5 | Score |
+--------+----+----+----+----+----+-------+
| 1      |    | 12 | 14 | 16 | 10 | 52    |
+--------+----+----+----+----+----+-------+
| 2      | 12 |    | 16 | 14 |  7 | 49    |
+--------+----+----+----+----+----+-------+
| 3      | 14 | 16 |    | 15 |  7 | 52    |
+--------+----+----+----+----+----+-------+
| 4      | 16 | 14 | 15 |    | 11 | 56    |
+--------+----+----+----+----+----+-------+
| 5      | 10 |  7 |  7 | 11 |    | 35    |
+--------+----+----+----+----+----+-------+


4

In [ ]:
for i, k in enumerate(data_p2):
    if k != kc:
        print(kc, "-", k)
        _, _ = needleman_wunsch(data_p2[kc], data_p2[k], display=1)
        print()

4 - 1
-1     - - 0                         G - 1                                 C - 2                                  C - 3                                          C - 4                          C - 5                         T - 6                                 A - 7
- - 0  {'val': 0, 'from': None}      {'val': -2, 'from': [0, 0]}           {'val': -4, 'from': [0, 1]}            {'val': -6, 'from': [0, 2]}                    {'val': -8, 'from': [0, 3]}    {'val': -10, 'from': [0, 4]}  {'val': -12, 'from': [0, 5]}          {'val': -14, 'from': [0, 6]}
C - 1  {'val': -2, 'from': [0, 0]}   {'val': -2, 'from': [[0, 0]]}         {'val': 0, 'from': [[1, 1], [0, 1]]}   {'val': 2, 'from': [[1, 2]]}                   {'val': 4, 'from': [[1, 3]]}   {'val': 6, 'from': [[1, 4]]}  {'val': 4, 'from': [[1, 5]]}          {'val': 2, 'from': [[1, 6]]}
G - 2  {'val': -4, 'from': [1, 0]}   {'val': 0, 'from': [[1, 1], [1, 0]]}  {'val': -2, 'from': [[1, 2], [2, 1]]}  {'val': 0, 'from': [[1, 3]]}        

In [ ]:
center = data_p2[4]

In [ ]:
# Center (S4) with S1
table, kc = needleman_wunsch(center, data_p2[1], s_match=2, s_nomatch=-1, display=1)

-1     - - 0                        G - 1                                 C - 2                                 C - 3                                 C - 4                         C - 5                          T - 6                                 A - 7
- - 0  {'val': 0, 'from': None}     {'val': -1, 'from': [0, 0]}           {'val': -2, 'from': [0, 1]}           {'val': -3, 'from': [0, 2]}           {'val': -4, 'from': [0, 3]}   {'val': -5, 'from': [0, 4]}    {'val': -6, 'from': [0, 5]}           {'val': -7, 'from': [0, 6]}
C - 1  {'val': -1, 'from': [0, 0]}  {'val': -1, 'from': [[0, 0]]}         {'val': 1, 'from': [[1, 1], [0, 1]]}  {'val': 3, 'from': [[1, 2]]}          {'val': 5, 'from': [[1, 3]]}  {'val': 7, 'from': [[1, 4]]}   {'val': 6, 'from': [[1, 5]]}          {'val': 5, 'from': [[1, 6]]}
G - 2  {'val': -2, 'from': [1, 0]}  {'val': 1, 'from': [[1, 1], [1, 0]]}  {'val': 0, 'from': [[1, 2], [2, 1]]}  {'val': 2, 'from': [[1, 3]]}          {'val': 4, 'from': [[1, 4]]}  {'val': 6,

In [ ]:
# Add gaps to center
center = "CGAGC---TTA"

# Center (S4) with S2
table, kc = needleman_wunsch(center, data_p2[2], s_match=2, s_nomatch=-1, display=1)

-1      - - 0                          C - 1                           T - 2                                  A - 3                                  G - 4                                 C - 5                                 C - 6                          T - 7                                 C - 8                                 A - 9
- - 0   {'val': 0, 'from': None}       {'val': -1, 'from': [0, 0]}     {'val': -2, 'from': [0, 1]}            {'val': -3, 'from': [0, 2]}            {'val': -4, 'from': [0, 3]}           {'val': -5, 'from': [0, 4]}           {'val': -6, 'from': [0, 5]}    {'val': -7, 'from': [0, 6]}           {'val': -8, 'from': [0, 7]}           {'val': -9, 'from': [0, 8]}
C - 1   {'val': -1, 'from': [0, 0]}    {'val': 2, 'from': [[0, 0]]}    {'val': 1, 'from': [[1, 1]]}           {'val': 0, 'from': [[1, 2]]}           {'val': -1, 'from': [[1, 3]]}         {'val': 1, 'from': [[1, 4]]}          {'val': 3, 'from': [[1, 5]]}   {'val': 2, 'from': [[1, 6]]}          {'val': 

# Bottom